In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from xulidulieu import Preimport warnings
warnings.filterwarnings("ignore")

In [2]:
CustomerData = "using python and tableau to processing data"  # Description of current skills
Field = "Data Science"  # Field of interest
Jobwant = "Data Analyst"  # Job title or role you want to apply for
CustomerID = "C00001"


In [3]:
df_job = pd.read_excel("Job.xlsx", sheet_name=Field)
df_skill = pd.read_excel("Skill.xlsx")

In [4]:
# Loại bỏ các hàng trùng lặp trong cột 'Skill'
df_skill = df_skill.drop_duplicates(subset='Skill', keep='first').reset_index(drop=True)

In [5]:
df_skill['Skill_Description'] = df_skill['Skill'] + " " + df_skill['Description'] + df_skill['Skill'] + df_skill['Skill']

In [6]:
df_job=Pre(df_job)
df_skill=Pre(df_skill)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_tfidf(df_skills):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_skills['Skill_Description'])
    return tfidf_matrix, tfidf_vectorizer

In [8]:
def recommend(customer_data, tfidf_matrix, tfidf_vectorizer, df_skill, threshold=0.0001):
    # Vectorize the input description
    customer_vector = tfidf_vectorizer.transform([customer_data])
    # Compute cosine similarity
    similarity_scores = cosine_similarity(customer_vector, tfidf_matrix).flatten()
    # Get relevant indices
    relevant_indices = np.where(similarity_scores > threshold)[0]
    # Sort by similarity score
    sorted_indices = relevant_indices[np.argsort(similarity_scores[relevant_indices])[::-1]]
    sorted_scores = similarity_scores[sorted_indices]
    # Get skill names
    skill_names = df_skill.iloc[sorted_indices]['Skill'].values
    return skill_names, sorted_scores


In [9]:
# Vectorize job descriptions for the selected field
tfidf_matrix, tfidf_vectorizer = vectorize_tfidf(df_skill)

# Call the recommend function to get top_skills and scores
top_skills, scores = recommend(CustomerData, tfidf_matrix, tfidf_vectorizer, df_skill)

# Create a DataFrame with the recommended skills and their similarity scores
recommended_skills_df = pd.DataFrame({
    'Skill': top_skills,
    'Similarity Score': scores
})

def cluster_data(df, n_clusters=2):
    try:
        # Prepare data for clustering
        X = df[['Similarity Score']]
        
        # Apply KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        df['Cluster'] = kmeans.fit_predict(X)
        
        # Split data into clusters
        cluster_0 = df[df['Cluster'] == 0]
        cluster_1 = df[df['Cluster'] == 1]
        
        return cluster_0, cluster_1
    except Exception as e:
        return df, df

# Perform clustering on the recommended skills
cluster_0_df, cluster_1_df = cluster_data(recommended_skills_df)

# Compare and return the cluster with the highest similarity score

# Display the cluster with the highest similarity score



In [10]:
def max_cluster(cluster_0_df, cluster_1_df):
    if cluster_0_df['Similarity Score'].max() > cluster_1_df['Similarity Score'].max():
        return cluster_0_df
    else:
        return cluster_1_df
greater_cluster = max_cluster(cluster_0_df, cluster_1_df)

In [11]:
def get_greater_cluster_list(greater_cluster):
    greater_cluster_list = greater_cluster['Skill'].tolist()
    return greater_cluster_list
List_greater_cluster = get_greater_cluster_list(greater_cluster)

In [12]:
def update_new_df(new_df, List_greater_cluster):
    for index, row in new_df.iterrows():
        for column in new_df.columns[1:]:  # Bỏ qua cột đầu tiên (Job Title)
            if column in List_greater_cluster:
                new_df.at[index, column] = 1
    return new_df
# chỉ dùng cho hàm 
# Cập nhật new_df
# job_skill_matrix = update_new_df(new_df, List_greater_cluster)

In [13]:
def update_new_df_row(new_df, i, List_greater_cluster):
    for ni in range(1, new_df.shape[1]):  # Bắt đầu từ cột thứ 2 (bỏ qua cột đầu tiên)
        new_df.iloc[i, ni] = 1 if new_df.columns[ni] in List_greater_cluster else 0
    return new_df

In [14]:
# Lặp qua từng phần tử trong cột cuối cùng của df_job
def process_job_descriptions(df_job, df_skill, tfidf_matrix, tfidf_vectorizer, threshold=0.0001):
    skills = df_skill['Skill'].tolist()

    # Tạo DataFrame mới với cột đầu tiên là cột 'Job Title' từ df_job
    new_df = df_job[['Job Title']].copy()

    # Thêm các cột kỹ năng vào DataFrame mới, khởi tạo giá trị mặc định là 0
    for skill in skills:
        new_df[skill] = 0
    for job_description,i in zip(df_job.iloc[:, -1],range(new_df.shape[0])):
        # Recommend skills based on the job description
        top_skills, scores = recommend(job_description, tfidf_matrix, tfidf_vectorizer, df_skill, threshold)
        
        # Create a DataFrame with the recommended skills and their similarity scores
        recommended_skills_df = pd.DataFrame({
            'Skill': top_skills,
            'Similarity Score': scores
        })
        
        # Perform clustering on the recommended skills
        cluster_0_df, cluster_1_df = cluster_data(recommended_skills_df)
        greater_cluster = max_cluster(cluster_0_df, cluster_1_df)
        List_greater_cluster = get_greater_cluster_list(greater_cluster)
        # Compare and return the cluster with the highest similarity score
        # Lấy danh sách các kỹ năng từ cột 'Skill' của df_skill
        update_new_df_row(new_df, i, List_greater_cluster)

        

        
        
    job_skill_matrix = new_df.copy() 
    return job_skill_matrix


In [15]:
job_skill_matrix=process_job_descriptions(df_job, df_skill, tfidf_matrix, tfidf_vectorizer, threshold=0.0001)

C:\Users\Hoang Duy\AppData\Local\Temp\ipykernel_11156\1295339563.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[skill] = 0
C:\Users\Hoang Duy\AppData\Local\Temp\ipykernel_11156\1295339563.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[skill] = 0
C:\Users\Hoang Duy\AppData\Local\Temp\ipykernel_11156\1295339563.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [16]:
# Truy vấn jobwant trong job_skill_matrix
jobwant_row = job_skill_matrix[job_skill_matrix['Job Title'] == Jobwant]


In [17]:
# Tạo DataFrame mới với cột đầu tiên là "Job Want"
# Define the skills list from the df_skill DataFrame
skills = df_skill['Skill'].tolist()

customer_skill_df = pd.DataFrame(columns=["Job Title"] + skills)

# Thêm một dòng dữ liệu với giá trị "Jobwant" ở cột đầu tiên và các cột kỹ năng khác bằng 0
customer_skill_df.loc[0] = [Jobwant] + [0] * len(skills)

# Cập nhật giá trị 1 cho các kỹ năng trong danh sách List_greater_cluster
Customer_skill_df=update_new_df_row(customer_skill_df, 0, List_greater_cluster)

In [18]:
# Ensure unique column names in both DataFrames

job_skill_matrix = job_skill_matrix.loc[:, ~job_skill_matrix.columns.duplicated()]

# Cập nhật cột 'Job Want' trong customer_skill_df
customer_skill_df['Job Title'] = customer_skill_df['Job Title'] + " " + CustomerID


# Merge customer_skill_df vào job_skill_matrix
job_skill_similarity = pd.concat([jobwant_row, customer_skill_df], ignore_index=True)

In [19]:
job_skill_similarity


,Job Title,Python,SQL,Machine Learning,Deep Learning,Data Visualization,Statistics,"Big Data Tools (Spark, Hadoop)",ETL/Data Pipeline,Business Knowledge,...,Visual Merchandising,Warehouse Operations,Web Performance Optimization,Wireframing,Teamwork,Time Management,Creativity,Leadership,Attention to Detail,Customer Focus
0,Data Analyst,1,1,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
1,Data Analyst C00001,1,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Lấy giá trị từ dòng 1 và dòng 2, bắt đầu từ cột thứ 2
row1 = job_skill_similarity.iloc[0, 1:]
row2 = job_skill_similarity.iloc[1, 1:]

# Tính hiệu giữa dòng 1 và dòng 2
difference = row1 - row2

# Lấy các cột có giá trị hiệu bằng 1
Miss_skill = difference[difference == 1].index.tolist()



In [21]:
Miss_skill

['SQL',
 'Data Visualization',
 'Excel',
 'Bioinformatics',
 'Time Series Analysis',
 'Data Cleaning',
 'Critical Thinking',
 'Forecasting',
 'Power BI',
 'Reporting Tools',
 'Statistical Analysis',
 'Trend Analysis',
 'Attention to Detail']